# OpenFOAM Potential Flow Visualization

This notebook provides interactive visualization and analysis of OpenFOAM potential flow simulation results around a circular cylinder.

## Features:
- Load VTK output files from OpenFOAM simulation
- Process and visualize velocity fields and streamlines
- Interactive analysis of potential flow characteristics
- Export publication-quality figures

## Prerequisites:
Ensure the OpenFOAM simulation has completed and VTK files are available in the `../VTK/` directory.

In [ ]:
# Install required visualization dependencies
%pip install -qq pyvista matplotlib scipy numpy

In [ ]:
# Import required libraries for data processing and visualization
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
from scipy.interpolate import griddata
from matplotlib.path import Path
from scipy.spatial import ConvexHull
import os
from pathlib import Path as FS

print("Libraries imported successfully!")

## Load and Process VTK Data

Load the potential flow solution and cylinder boundary data from the OpenFOAM simulation results.

In [ ]:
# ============================================================================
# LOAD AND PROCESS VTK DATA FROM POTENTIALFOAM SIMULATION
# ============================================================================

# Check if VTK directory exists
vtk_path = FS('../VTK')
if not vtk_path.exists():
    print("ERROR: VTK directory not found. Please run the OpenFOAM simulation first.")
    print("Expected path: ../VTK/")
else:
    print(f"Found VTK directory: {vtk_path.absolute()}")
    
    # Load the potential flow solution and cylinder boundary
    run = next(d for d in vtk_path.iterdir() if d.is_dir())  # first sub-folder
    print(f"Loading data from: {run}")
    
    mesh = pv.read(run / 'internal.vtu')
    cylinder = pv.read(run / 'boundary' / 'cylinder.vtp')
    
    print(f"Mesh loaded: {mesh.n_cells} cells, {mesh.n_points} points")
    print(f"Cylinder boundary loaded: {cylinder.n_points} points")
    
    # Extract cell centers and velocity data from the potential flow solution
    cells = mesh.cell_centers()
    centres = cells.points[:, :2]          # 2D cell-center coordinates (N×2)
    U = mesh.cell_data["U"][:, :2]         # Velocity components from potential flow [u, v] (N×2)
    
    print(f"Extracted {len(centres)} cell centers and velocity vectors")

## Create Cylinder Boundary Mask

Extract the cylinder boundary and create a mask to exclude interior points from the visualization.

In [ ]:
# ============================================================================
# CREATE CYLINDER BOUNDARY POLYGON FOR MASKING
# ============================================================================

# Extract 2D cylinder boundary points and create convex hull
c2d = cylinder.points[:, :2]
hull = ConvexHull(c2d)
poly = c2d[hull.vertices]              # Ordered boundary vertices
poly_path = Path(poly)                 # Create matplotlib Path for inside/outside tests

print(f"Cylinder boundary: {len(poly)} vertices")

# Remove cell centers that are inside the cylinder body
keep = ~poly_path.contains_points(centres)
centres, U = centres[keep], U[keep]

print(f"Filtered to {len(centres)} external points (removed {np.sum(~keep)} interior points)")

## Interpolate to Regular Grid

Convert the unstructured CFD data to a regular grid for streamline visualization.

In [ ]:
# ============================================================================
# INTERPOLATE TO REGULAR GRID FOR STREAMLINE VISUALIZATION
# ============================================================================

# Create regular rectangular grid for streamline plotting
nx = ny = 300  # Grid resolution
xi = np.linspace(centres[:,0].min(), centres[:,0].max(), nx)
yi = np.linspace(centres[:,1].min(), centres[:,1].max(), ny)
X, Y = np.meshgrid(xi, yi)

print(f"Created {nx}×{ny} regular grid")
print(f"X range: [{xi.min():.2f}, {xi.max():.2f}]")
print(f"Y range: [{yi.min():.2f}, {yi.max():.2f}]")

# Interpolate velocity components from unstructured to structured grid
u = griddata(centres, U[:,0], (X, Y), method='linear')  # x-velocity component
v = griddata(centres, U[:,1], (X, Y), method='linear')  # y-velocity component

# Apply cylinder mask to grid (remove points inside cylinder)
inside = poly_path.contains_points(np.c_[X.ravel(), Y.ravel()]).reshape(X.shape)
u[inside] = v[inside] = np.nan  # Set interior points to NaN
speed = np.hypot(u, v)              # Calculate velocity magnitude

print(f"Interpolation complete. Speed range: [{np.nanmin(speed):.3f}, {np.nanmax(speed):.3f}] m/s")

## Streamline Visualization

Create the main streamline plot showing smooth potential flow patterns around the cylinder.

In [ ]:
# ============================================================================
# CREATE POTENTIAL FLOW STREAMLINE VISUALIZATION
# ============================================================================

# Create streamline plot showing smooth potential flow patterns
plt.figure(figsize=(12, 8))
strm = plt.streamplot(X, Y, u, v, 
                     color=speed,           # Color streamlines by velocity magnitude
                     cmap='viridis',        # Use viridis colormap
                     density=1.5,           # Streamline density
                     linewidth=1.2)         # Line thickness

# Add colorbar for velocity magnitude
cbar = plt.colorbar(strm.lines, label='Velocity Magnitude (m/s)')
cbar.ax.tick_params(labelsize=10)

# Draw cylinder outline
poly_closed = np.vstack([poly, poly[0]])   # Close the polygon
plt.fill(poly_closed[:,0], poly_closed[:,1], 'white', edgecolor='black', linewidth=2, zorder=10)

# Format plot for clean presentation
plt.axis('equal')      # Equal aspect ratio to preserve geometry
plt.xlabel('X Position (m)', fontsize=12)
plt.ylabel('Y Position (m)', fontsize=12)
plt.title('Potential Flow Around Cylinder\n(Smooth streamlines, no separation)', 
          fontsize=14, pad=20)
plt.grid(True, alpha=0.3)
plt.tight_layout()     # Optimize layout
plt.show()

print("\nStreamline visualization complete!")
print("\nKey characteristics of potential flow:")
print("• Smooth streamlines with no flow separation")
print("• Perfect fore-aft symmetry around the cylinder")
print("• No boundary layers (inviscid flow assumption)")
print("• Streamlines compress around cylinder, increasing velocity")

## Additional Analysis Options

Explore different visualization options and analysis techniques.

In [ ]:
# Create velocity magnitude contour plot
plt.figure(figsize=(12, 8))
contour = plt.contourf(X, Y, speed, levels=20, cmap='plasma')
plt.colorbar(contour, label='Velocity Magnitude (m/s)')

# Draw cylinder outline
plt.fill(poly_closed[:,0], poly_closed[:,1], 'white', edgecolor='black', linewidth=2)

plt.axis('equal')
plt.xlabel('X Position (m)', fontsize=12)
plt.ylabel('Y Position (m)', fontsize=12)
plt.title('Velocity Magnitude Contours', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Create pressure coefficient analysis (if pressure data is available)
try:
    # Extract pressure data if available
    p = mesh.cell_data.get("p")
    if p is not None:
        print("Pressure data found - creating pressure coefficient plot")
        
        # Calculate pressure coefficient: Cp = (p - p_inf) / (0.5 * rho * U_inf^2)
        # For potential flow: Cp = 1 - (U/U_inf)^2
        U_inf = np.nanmean(speed[~np.isnan(speed)])  # Estimate freestream velocity
        Cp_theoretical = 1 - (speed / U_inf)**2
        
        plt.figure(figsize=(12, 8))
        cp_contour = plt.contourf(X, Y, Cp_theoretical, levels=20, cmap='RdBu_r', vmin=-3, vmax=1)
        plt.colorbar(cp_contour, label='Pressure Coefficient Cp')
        
        # Draw cylinder outline
        plt.fill(poly_closed[:,0], poly_closed[:,1], 'white', edgecolor='black', linewidth=2)
        
        plt.axis('equal')
        plt.xlabel('X Position (m)', fontsize=12)
        plt.ylabel('Y Position (m)', fontsize=12)
        plt.title('Pressure Coefficient Distribution', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print("No pressure data found in the mesh")
except Exception as e:
    print(f"Could not create pressure analysis: {e}")

## Export Results

Save high-quality figures for reports or publications.

In [ ]:
# Export high-resolution streamline plot
plt.figure(figsize=(12, 8), dpi=300)
strm = plt.streamplot(X, Y, u, v, 
                     color=speed, cmap='viridis', density=1.5, linewidth=1.2)
plt.colorbar(strm.lines, label='Velocity Magnitude (m/s)')
plt.fill(poly_closed[:,0], poly_closed[:,1], 'white', edgecolor='black', linewidth=2)
plt.axis('equal')
plt.xlabel('X Position (m)', fontsize=12)
plt.ylabel('Y Position (m)', fontsize=12)
plt.title('Potential Flow Around Cylinder', fontsize=14, pad=20)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
output_file = '../potential_flow_streamlines.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
print(f"High-resolution plot saved to: {output_file}")
plt.show()

## Summary

This visualization notebook demonstrates the key characteristics of potential flow around a circular cylinder:

1. **Smooth streamlines** - No flow separation or vorticity
2. **Symmetrical flow** - Perfect fore-aft symmetry
3. **No boundary layers** - Inviscid flow assumption allows slip at walls
4. **Velocity acceleration** - Flow accelerates around the cylinder

The potential flow solution provides valuable insight into ideal fluid behavior and serves as a baseline for more complex viscous flow analysis.